# Chain

## ConversationChain

In [2]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains.conversation.base import ConversationChain

chat = ChatOpenAI(model='gpt-3.5-turbo-0125')
memory = ConversationBufferMemory()
chain = ConversationChain(
    llm=chat,
    memory=memory,
    verbose=True
)

chain.predict(input='Oi')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Oi
AI:

> Finished chain.


'Hello! How can I assist you today?'

In [3]:
chain.run('Oi')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Oi
AI: Hello! How can I assist you today?
Human: Oi
AI:

> Finished chain.


'Hello again! How can I help you today?'

In [7]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
'''
Essa é uma conversa amigável entre um humano e uma IA.

Conversa atual:

{history}
Human: {input}
Ai:''')

chat = ChatOpenAI(model='gpt-3.5-turbo-0125')
memory = ConversationBufferMemory()
chain = ConversationChain(
    prompt=prompt_template,
    llm=chat,
    memory=memory,
    verbose=True
)

chain.predict(input='Oi')




> Entering new ConversationChain chain...
Prompt after formatting:

Essa é uma conversa amigável entre um humano e uma IA.

Conversa atual:


Human: Oi
Ai:

> Finished chain.


'Oi! Como posso te ajudar hoje?'

In [8]:
chain.predict(input='Oi de novo')



> Entering new ConversationChain chain...
Prompt after formatting:

Essa é uma conversa amigável entre um humano e uma IA.

Conversa atual:

Human: Oi
AI: Oi! Como posso te ajudar hoje?
Human: Oi de novo
Ai:

> Finished chain.


'Olá! Estou aqui para conversar ou ajudar com qualquer dúvida que você tenha. Como posso ser útil hoje?'

## LLMChain

In [9]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains.llm import LLMChain

chat = ChatOpenAI(model='gpt-3.5-turbo-0125')

In [12]:
prompt = PromptTemplate.from_template(
'''Qual o melhor nome de empresa para 
uma empresa que desenvolve o produto: {produto}.
Retorne apenas um nome.'''
)

chain = LLMChain(llm=chat, prompt=prompt)

produto = 'Copos de vidro inquebráveis'
chain.run(produto)

'VidroShield'

## SimpleSequentialChain

In [17]:
from langchain.chains import SimpleSequentialChain

chat = ChatOpenAI(model='gpt-3.5-turbo-0125')

In [18]:
prompt = PromptTemplate.from_template(
'''Qual o melhor nome de empresa para 
uma empresa que desenvolve o produto: {produto}.
Retorne apenas um nome.'''
)

chain_nome = LLMChain(llm=chat, prompt=prompt)

prompt_descricao = PromptTemplate.from_template(
'''Dado a empresa com o seguinte nome: {nome_empresa}.
Dê uma descrição de até 50 palavras das atividades
dessa empresa.'''
)

chain_descricao = LLMChain(llm=chat, prompt=prompt_descricao)



In [19]:
chain = SimpleSequentialChain(
    chains=[chain_nome, chain_descricao],
    verbose=True
)
produto = 'Copos de vidro inquebráveis'
chain.run(produto)



> Entering new SimpleSequentialChain chain...
Vidro Resistente Ltda.
A Vidro Resistente Ltda. é uma empresa especializada na fabricação e comercialização de produtos de vidro de alta resistência, como vidros blindados, temperados e laminados. Atendendo às necessidades de segurança e proteção, oferece soluções inovadoras e de qualidade para diversos segmentos do mercado.

> Finished chain.


'A Vidro Resistente Ltda. é uma empresa especializada na fabricação e comercialização de produtos de vidro de alta resistência, como vidros blindados, temperados e laminados. Atendendo às necessidades de segurança e proteção, oferece soluções inovadoras e de qualidade para diversos segmentos do mercado.'

## SequentialChain

não precisa necessáriamente ser sequencial pode ser parelela

In [22]:
from langchain.chains import SequentialChain

chat = ChatOpenAI(model='gpt-3.5-turbo-0125')

In [26]:
prompt = PromptTemplate.from_template(
'''Qual o melhor nome de empresa para 
uma empresa que desenvolve o produto: {produto}.
Retorne apenas um nome.'''
)

chain_nome = LLMChain(llm=chat, prompt=prompt, output_key='nome_empresa')

prompt_descricao = PromptTemplate.from_template(
'''Dado a empresa com o seguinte nome: {nome_empresa}
e que produz o seguinte produto {produto}.
Dê uma deswcrição de até 50 palavras das atividades
dessa empresa.'''
)

chain_descricao = LLMChain(llm=chat, prompt=prompt_descricao, output_key='descricao_empresa')

prompt_traducao = PromptTemplate.from_template(
'''Crie um nome em inglês para a empresa de nome {nome_empresa}
que possui a seguinte descrição {descricao_empresa}'''
)

chain_traducao = LLMChain(llm=chat, prompt=prompt_traducao, output_key='nome_ingles', )


In [28]:
chain = SequentialChain(
    chains=[chain_nome, chain_descricao, chain_traducao],
    input_variables=['produto'],
    output_variables=['nome_empresa', 'descricao_empresa', 'nome_ingles'],
    verbose=True
)
produto = 'Copos de vidro inquebráveis'
resposta = chain.invoke({'produto': produto})
resposta



> Entering new SequentialChain chain...

> Finished chain.


{'produto': 'Copos de vidro inquebráveis',
 'nome_empresa': 'Vidro Resistente Inc.',
 'descricao_empresa': 'A Vidro Resistente Inc. é uma empresa especializada na fabricação de copos de vidro inquebráveis, oferecendo durabilidade e resistência para o dia a dia dos consumidores. Com tecnologia avançada e materiais de alta qualidade, nossos produtos garantem segurança e praticidade para o uso doméstico e comercial.',
 'nome_ingles': 'Unbreakable Glassware Co.'}

In [31]:
resposta['descricao_empresa']

'A Vidro Resistente Inc. é uma empresa especializada na fabricação de copos de vidro inquebráveis, oferecendo durabilidade e resistência para o dia a dia dos consumidores. Com tecnologia avançada e materiais de alta qualidade, nossos produtos garantem segurança e praticidade para o uso doméstico e comercial.'

### Mais um exemplo

In [32]:
from langchain.chains.sequential import SequentialChain

chat = ChatOpenAI(model="gpt-3.5-turbo-0125")

prompt_correcao = """
Determine se a solução do aluno está correta ou não.

Pergunta:
Estou construindo uma instalação de energia solar e preciso de ajuda com as contas de custos as finanças.
- O terreno custa R$ 100 / metro quadrado
- Posso comprar painéis solares por R$ 250 / metro quadrado
- Negociei um contrato de manutenção que me custará R$ 100 mil por ano e  R$ 10 adicionais por metro quadrado

Qual é o custo total para o primeiro ano de operações
em função do número de metros quadrados.

Solução do aluno:
Seja x o tamanho da instalação em metros quadrados.
Custos:
1. Custo do terreno: 100x
2. Custo do painel solar: 250x
3. Custo de manutenção: 100.000 + 100x
Custo total: 100x + 250x + 100.000 + 100x = 450x + 100.000
"""

chat.invoke(prompt_correcao)

AIMessage(content='A solução do aluno está correta. O custo total para o primeiro ano de operações em função do número de metros quadrados é de 450x + 100.000.', response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 243, 'total_tokens': 282}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-f226058a-a3a3-4ec9-bd99-c421b8bcf153-0')

In [39]:
prompt_correcao = """
Determine se a solução do aluno está correta ou não.

Pergunta:
Estou construindo uma instalação de energia solar e preciso de ajuda com as contas de custos as finanças.
- O terreno custa R$ 100 / metro quadrado
- Posso comprar painéis solares por R$ 250 / metro quadrado
- Negociei um contrato de manutenção que me custará R$ 100 mil por ano e  R$ 10 adicionais por metro quadrado

Qual é o custo total para o primeiro ano de operações
em função do número de metros quadrados.

Solução do aluno:
Seja x o tamanho da instalação em metros quadrados.
Custos:
1. Custo do terreno: 100x
2. Custo do painel solar: 250x
3. Custo de manutenção: 100.000 + 100x
Custo total: 100x + 250x + 100.000 + 100x = 450x + 100.000
"""


prompt = PromptTemplate.from_template(
'''Resolva a questão fornecida para você:

{questao}

Responda com calma e passo a passo.'''
)

chain_solucao = LLMChain(llm=chat, prompt=prompt, output_key='solucao', verbose=True)

prompt = PromptTemplate.from_template(
'''
======
Dado o seguinte problema:
{questao}
======

Aqui está a solução correta:

{solucao}

======

Aqui está a solução de um aluno:

{solucao_aluno}

=====

Se baseando apenas na solução correta, verifique se o aluno
respondeu corretamente.
'''
)

chain_final = LLMChain(llm=chat, prompt=prompt, output_key='resposta_final', verbose=True)

chain = SequentialChain(
    chains=[chain_solucao, chain_final],
    input_variables=["questao", "solucao_aluno"],
    output_variables=["solucao", "resposta_final"],
    verbose=True
)

In [40]:
questao = '''
Estou construindo uma instalação de energia solar e preciso de ajuda com as contas de custos as finanças.
- O terreno custa R$ 100 / metro quadrado
- Posso comprar painéis solares por R$ 250 / metro quadrado
- Negociei um contrato de manutenção que me custará R$ 100 mil por ano e  R$ 10 adicionais por metro quadrado

Qual é o custo total para o primeiro ano de operações
em função do número de metros quadrados.
'''

solucao_aluno = '''
Seja x o tamanho da instalação em metros quadrados.
Custos:
1. Custo do terreno: 100x
2. Custo do painel solar: 250x
3. Custo de manutenção: 100.000 + 100x
Custo total: 100x + 250x + 100.000 + 100x = 450x + 100.000
'''

chain.invoke({'questao': questao, 'solucao_aluno': solucao_aluno})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Resolva a questão fornecida para você:


Estou construindo uma instalação de energia solar e preciso de ajuda com as contas de custos as finanças.
- O terreno custa R$ 100 / metro quadrado
- Posso comprar painéis solares por R$ 250 / metro quadrado
- Negociei um contrato de manutenção que me custará R$ 100 mil por ano e  R$ 10 adicionais por metro quadrado

Qual é o custo total para o primeiro ano de operações
em função do número de metros quadrados.


Responda com calma e passo a passo.

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

Dado o seguinte problema:

Estou construindo uma instalação de energia solar e preciso de ajuda com as contas de custos as finanças.
- O terreno custa R$ 100 / metro quadrado
- Posso comprar painéis solares por R$ 250 / metro quadrado
- Negociei um contrato de manutenção que me custará R$ 100 mil por ano e  R$ 10 adicionais

{'questao': '\nEstou construindo uma instalação de energia solar e preciso de ajuda com as contas de custos as finanças.\n- O terreno custa R$ 100 / metro quadrado\n- Posso comprar painéis solares por R$ 250 / metro quadrado\n- Negociei um contrato de manutenção que me custará R$ 100 mil por ano e  R$ 10 adicionais por metro quadrado\n\nQual é o custo total para o primeiro ano de operações\nem função do número de metros quadrados.\n',
 'solucao_aluno': '\nSeja x o tamanho da instalação em metros quadrados.\nCustos:\n1. Custo do terreno: 100x\n2. Custo do painel solar: 250x\n3. Custo de manutenção: 100.000 + 100x\nCusto total: 100x + 250x + 100.000 + 100x = 450x + 100.000\n',
 'solucao': 'Vamos calcular o custo total para o primeiro ano de operações em função do número de metros quadrados.\n\nVamos chamar o número de metros quadrados de "x".\n\n1. Custo do terreno: R$ 100 por metro quadrado\nCusto total do terreno = R$ 100 * x\n\n2. Custo dos painéis solares: R$ 250 por metro quadrado\n